For interacting with the new template format while drafting proposals.

In [1]:
#%%
import os
import re
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import requests
import xlwings as xw

import functions
import hide

In [2]:
# Connect to work book.
file = "J12234 MODEC - EQUINOR BACALHAU FPSO - VSAT.xlsx"
# file = '01 Costing sheet V3.67 Rev.0.xlsm'
downloads_folder = os.path.join(os.path.expanduser('~'), 'Downloads')
file_path = Path(downloads_folder, file)
wb = xw.Book(file_path, password=hide.legacy)

In [11]:
def fill_formula(sheet):
    last_row = sheet.range('C100000').end('up').row
    sheet.range('N6:N' + str(last_row)).formula = '=IF(K6<>"",K6*(1-M6),"")'
    sheet.range('O6:O' + str(last_row)).formula = '=IF(AND(D6<>"", K6<>"",H6<>"OPTION"),D6*N6,"")'
    # Exchange rates
    sheet.range('Q6:Q' + str(last_row)).formula = '=IF(Config!B12="SGD",IF(J6<>"",VLOOKUP(J6,Config!$A$2:$B$10,2,FALSE),""),IF(J6<>"",VLOOKUP(J6,Config!$A$2:$B$10,2,FALSE)/VLOOKUP(Config!$B$12,Config!$A$2:$B$10,2,FALSE),""))'
    sheet.range('R6:R' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>"") ,N6*Q6,"")'
    sheet.range('S6:S' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>"",H6<>"OPTION") ,D6*R6,"")'
    sheet.range('T6:T' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""), (R6*(1+$K$1+$K$2+$K$3+$K$4))/(1-0.05),"")'
    # Below formula does not capture all options yet
    sheet.range('U6:U' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),IF(H6="OPTION","",D6*T6),"")'
    sheet.range('V6:V' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),R6*$K$1,"")'
    sheet.range('W6:W' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),R6*$K$2,"")'
    sheet.range('X6:X' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),R6*$K$3,"")'
    sheet.range('Y6:Y' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),R6*$K$4,"")'
    sheet.range('Z6:Z' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),T6-(R6+V6+W6+X6+Y6),"")'
    sheet.range('AA6:AA' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),$I$2,"")'
    sheet.range('AC6:AC' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),CEILING(T6/(1-AA6), 1),"")'
    sheet.range('AD6:AD' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>"", H6<>"OPTION",H6<>"INCLUDED"),D6*AC6,"")'
    sheet.range('AE6:AE' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>""),IF(AB6<>"",AB6,AC6),"")'
    sheet.range('AF6:AF' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>"", H6<>"OPTION", H6<>"INCLUDED"),D6*AE6,"")'
    sheet.range('AG6:AG' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>"", H6<>"OPTION", H6<>"INCLUDED"),AF6-U6,"")'
    sheet.range('AH6:AH' + str(last_row)).formula = '=IF(AND(AG6<>"",AG6<>0),AG6/AF6,"")'
    sheet.range('F6:F' + str(last_row)).formula = '=IF(AE6<>"", AE6,"")'
    sheet.range('G6:G' + str(last_row)).formula = '=IF(AND(F6<>"", H6<>"OPTION", H6<>"INCLUDED"), D6*F6,"")'
    sheet.range('L6:L' + str(last_row)).formula = '=IF(AND(D6<>"",K6<>"",H6<>"OPTION"),D6*K6,"")'
    (wb.sheets['Config'].range('100:100')).copy(sheet.range(str(last_row+2) + ':' + str(last_row+2)))
    sheet.range('F'+ str(last_row+2)).formula = '="Subtotal (" & Config!B12 & ")"'
    sheet.range('F'+ str(last_row+2)).font.bold = True
    sheet.range('G' + str(last_row+2)).formula = '=SUM(G6:G' + str(last_row+1) + ')'
    sheet.range('G' + str(last_row+2)).font.bold = True
    sheet.range('U' + str(last_row+2)).formula = '=SUM(U6:U' + str(last_row+1) + ')'
    sheet.range('U' + str(last_row+2)).font.bold = True
    sheet.range('AF' + str(last_row+2)).formula = '=SUM(AF6:AF' + str(last_row+1) + ')'
    sheet.range('AF' + str(last_row+2)).font.bold = True
    sheet.range('AG' + str(last_row+2)).formula = '=SUM(AG6:AG' + str(last_row+1) + ')'
    sheet.range('AG' + str(last_row+2)).font.bold = True
    sheet.range('AH' + str(last_row+2)).formula = '=AG' + str(last_row+2) + '/AF' + str(last_row+2)
    sheet.range('AH' + str(last_row+2)).font.bold = True

In [14]:
fill_formula(wb.sheets.active)